In [ ]:
# FlannBasedMacher() 예제 9.4-2 프로그램

import numpy as np
import cv2

img1 = cv2.imread('box.png', 0)  # query Image 로드
img2 = cv2.imread('box_in_scene.png', 0) # train Image 로드

# Find the keypoints and descriptors with SIFT
sift = cv2.xfeatures2d.SIFT_create() 
keyPoints1, descriptors1 = sift.detectAndCompute(img1, None)
keyPoints2, descriptors2 = sift.detectAndCompute(img2, None)

# 인덱스 파라미터와 검색 파라미터 설정 
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=30)

# Flann 매처 생성, sorted = Ture(기본값): 정렬해서 반환
matcher = cv2.FlannBasedMatcher(index_params, search_params)

# 가장 가까운 매칭점 2개씩 검색하여 리스트로 반환
matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

# 가장 유사한 매칭점이 다른 후보와 확연한 차이가 날 경우
# 우수한 매칭점들을 저장할 리스트 생성
goodMatches = []     
for m, n in matches:   
    if m.distance < 0.7 * n.distance:   
        goodMatches.append(m)           

print(), print('num of goodMatches =', len(goodMatches))
print('queryIdx trainIdx distance:')
for i in range(5):
    print('%5d'%goodMatches[i].queryIdx, '%8d'%goodMatches[i].trainIdx,
          '%10.2f'%goodMatches[i].distance)
       
# 거리값을 기준으로 정렬
sorted_GMatches = sorted(goodMatches, key = lambda x : x.distance)

print(), print('queryIdx trainIdx distance')
for i in range(5):
    print('%5d'%sorted_GMatches[i].queryIdx, 
          '%8d'%sorted_GMatches[i].trainIdx,
          '%10.2f'%sorted_GMatches[i].distance)

MIN_MATCH_COUNT = 10 # 최소한 검출되어야 하는 매칭점의 개수 세팅
if len(goodMatches) > MIN_MATCH_COUNT :      
    # 매칭점들을 연결해 두 이미지 간의 매칭 관계 그리기
    drawParas = dict(matchColor=(0,255,0),singlePointColor=None,flags=2)
    result = cv2.drawMatches(img1, keyPoints1, img2, keyPoints2, 
                             sorted_GMatches[:15], None, **drawParas)
    
    # Display the results
    cv2.imshow('FLANN result', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else :  # 매칭점이 충분히 검출되지 않았을 경우
    print("Not enough matches are found ")

